In [0]:
import pandas as pd
import numpy as np

from importlib.machinery import SourceFileLoader
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import cosine_similarity

# DI = SourceFileLoader("DI", "/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/6.Source/src/DI.py").load_module()

# Handler = DI.Handler.Handler()
# Graph = DI.Graph.Graph()
# AlgoML = DI.AlgoML.AlgoML()
# Refine = DI.Refine.Refine()
# Metrics = DI.Metrics.Metrics()

client_one = pd.read_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/1.Original/estaticos_portfolio1.csv")
# client_two = pd.read_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/1.Original/estaticos_portfolio2.csv")
# client_three = pd.read_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/1.Original/estaticos_portfolio3.csv")
market = pd.read_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/1.Original/estaticos_market.csv")
_market = market.copy()

In [2]:
market = market.astype('category')
market.dtypes

Unnamed: 0                 category
id                         category
fl_matriz                  category
de_natureza_juridica       category
sg_uf                      category
                             ...   
qt_funcionarios_24meses    category
tx_crescimento_12meses     category
tx_crescimento_24meses     category
tx_rotatividade            category
qt_filiais                 category
Length: 182, dtype: object

In [0]:
def getFromMarket(market, df):
  try:
    result = pd.merge(market, df, on="id", how="inner")
    if(result.shape[0] != 0):
      return result
    else:
      return "Not found the id's collection present between market and dataseet"
  except Exception as e:
    return str(e)

def imputerCategorical(df, features):
  df.loc[:, features] = SimpleImputer(fill_value = "SEM INFORMACAO", strategy='constant').fit_transform(df[features])

def oneEncoder(df):
  categorical_df = df.loc[:, ['de_ramo', 'setor', 'nm_divisao', 'nm_segmento',"natureza_juridica_macro",
                                        "de_natureza_juridica", "fl_optante_simples", 'de_faixa_faturamento_estimado',
                                        'de_faixa_faturamento_estimado_grupo', 'idade_emp_cat']].columns
  dummy = None
  start = False
  for categorical in list(categorical_df):
    if(not start):
      dummy = pd.get_dummies(df[categorical], columns=categorical, prefix=f"d_{categorical}_")
      start = True
    else:
      _dummy = pd.get_dummies(df[categorical], columns=categorical, prefix=f"d_{categorical}_")
      dummy = dummy.join(_dummy)

  return dummy

In [0]:
def PiPe(df_x, df_y):
  features_ramo = [
    "de_ramo",
    "setor",
    "nm_divisao",
    "nm_segmento",
  ]

  features_burocracia = [
      "natureza_juridica_macro",
      "de_natureza_juridica",
      "fl_optante_simples"
  ]

  features_consolidada = [
      "de_faixa_faturamento_estimado", 
      "de_faixa_faturamento_estimado_grupo",
      "idade_emp_cat"
  ]

  features = features_ramo + features_burocracia + features_consolidada

  try:
    #safe imutable because of the shallow copy from param in python
    df_y = getFromMarket(df_x, df_y["id"])
    # return df_y
  except Exception as e:
    return f"[PIPE 0] ERROR -> \n {str(e)}"
  
  try:
    #safe imutable because of the shallow copy from param in python
    df_y = df_y[features]
    df_x = df_x[features]
  except Exception as e:
    return f"[PIPE 1] ERROR -> \n {str(e)}"

  try:
    #void functions
    imputerCategorical(df_x, features)
    imputerCategorical(df_y, features)
  except Exception as e:
    return f"[PIPE 2] ERROR -> \n {str(e)}"

  try:
    df_x = oneEncoder(df_x)
    df_y = oneEncoder(df_y)
  except Exception as e:
    return f"[PIPE 3] ERROR -> \n {str(e)}"

  return [df_x, df_y]

In [0]:
#TODO: por enquanto o y ta sendo individual, é preciso somar a linha individual de cada row, e entrega aquela com maior soma
#ALTTODO: (462298, 1) esse é o shape de coss, representando a porcetagem em cada index presnete em x, é possível loopar todo x, salva num objeto o index 
# e ver aquele que mais aparece para todo y
def recommendation(x, y, nivel = 1, limite = 1):
  indexes_all = {}
  
  for pos in range(0, y.shape[0]):
    coss = cosine_similarity(x[y.columns], [y.iloc[pos]])
    coss_serie = pd.Series(coss[:,0])
    
    if(nivel == 1):
      indexes_array = coss_serie[coss_serie == 1].index
    else:
      indexes_array = coss_serie[(coss_serie >= nivel) & (coss_serie < limite)].index
      
    for idx in indexes_array:
      if idx in indexes_all:
        indexes_all[idx] += 1
      else:
        indexes_all[idx] = 0
  
  return indexes_all
  # recomended = x.iloc[indexes]

In [0]:
# def iter_indexes_array(array_idx):
#   pos_actual_d = 0
#   pos_final_d = len(array_idx)

#   while(pos_actual_d != pos_final_d):
#     hash_key = array_idx[pos_actual_d];

#     if array_idx[pos_actual_d] in indexes_all:
#       indexes_all[hash_key] += 1
#     else:
#       indexes_all[hash_key] = 0
    
#     pos_actual_d += 1
#     yield pos_actual_d

# def _iter_fillIndexesDict(x, y, offset):
#   pos_actual = offset
#   pos_final = y.shape[0]

#   while(pos_actual != pos_final):
#     coss = cosine_similarity(x[y.columns], [y.iloc[pos_actual]])
#     coss_serie = pd.Series(coss[:,0])
    
#     del coss

#     indexes_array = coss_serie[coss_serie == 1].index
#     del coss_serie
    
#     for __idx in iter_indexes_array(indexes_array):
#       __idx
    
#     pos_actual += 1
#     yield pos_actual

# indexes_all = {}
# def recommendation(x, y, offset = 2):

#   print("[RECOMMENDATION] going to iterador...")
#   for _iter in _iter_fillIndexesDict(x, y, offset):
#     _iter

#   return indexes_all

In [0]:
def getRecommendedFromMarket(market, dict_recommended):
  features_ramo = [
    "de_ramo",
    "setor",
    "nm_divisao",
    "nm_segmento",
  ]

  features_burocracia = [
      "natureza_juridica_macro",
      "de_natureza_juridica",
      "fl_optante_simples"
  ]

  features_consolidada = [
      "de_faixa_faturamento_estimado", 
      "de_faixa_faturamento_estimado_grupo",
      "idade_emp_cat"
  ]

  features = features_ramo + features_burocracia + features_consolidada
  
  dict_recommended = {k: v for k, v in sorted(dict_recommended.items(), key = lambda x: x[1], reverse=True)}

  recommended = market[features]

  return recommended.iloc[list(dict_recommended.keys())]

In [7]:
x, y = PiPe(market, client_two)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
%%time
obj_recommended = recommendation(x, y)

CPU times: user 6min 28s, sys: 1min, total: 7min 29s
Wall time: 5min 57s


In [9]:
%%time
end = getRecommendedFromMarket(market, obj_recommended)

CPU times: user 7.99 ms, sys: 0 ns, total: 7.99 ms
Wall time: 10.9 ms


In [10]:
end

,de_ramo,setor,nm_divisao,nm_segmento,natureza_juridica_macro,de_natureza_juridica,fl_optante_simples,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,idade_emp_cat
19833,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
45821,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
224971,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
225637,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
285529,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
301929,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
307965,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
313372,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
335999,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
340402,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES EMPRESARIAIS,SOCIEDADE EMPRESARIA LIMITADA,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20


In [76]:
x, y = PiPe(market, client_one)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [98]:
%%time
obj_recommended = recommendation(x, y, 0, 0.4)

CPU times: user 3min 13s, sys: 55.6 s, total: 4min 8s
Wall time: 2min 54s


In [99]:
%%time
end = getRecommendedFromMarket(market, obj_recommended)

CPU times: user 721 ms, sys: 3.96 ms, total: 725 ms
Wall time: 727 ms


In [100]:
end

,de_ramo,setor,nm_divisao,nm_segmento,natureza_juridica_macro,de_natureza_juridica,fl_optante_simples,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,idade_emp_cat
123,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,OUTROS,MUNICIPIO,False,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",> 20
2049,SERVICOS FINANCEIROS,SERVIÇO,ATIVIDADES DE SERVICOS FINANCEIROS,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVICOS R...,ENTIDADES EMPRESARIAIS,SOCIEDADE ANONIMA ABERTA,False,"DE R$ 1.500.000,01 A R$ 4.800.000,00",ACIMA DE 1 BILHAO DE REAIS,5 a 10
3120,SERVICOS FINANCEIROS,SERVIÇO,ATIVIDADES DE SERVICOS FINANCEIROS,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVICOS R...,ENTIDADES EMPRESARIAIS,EMPRESA PUBLICA,False,"DE R$ 10.000.000,01 A R$ 30.000.000,00",ACIMA DE 1 BILHAO DE REAIS,> 20
4461,SERVICOS DE EDUCACAO,SERVIÇO,EDUCACAO,EDUCACAO,ENTIDADES SEM FINS LUCRATIVOS,ASSOCIACAO PRIVADA,False,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",> 20
4643,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,ARMAZENAMENTO E ATIVIDADES AUXILIARES DOS TRAN...,TRANSPORTE ARMAZENAGEM E CORREIO,ENTIDADES EMPRESARIAIS,EMPRESA PUBLICA,False,"DE R$ 4.800.000,01 A R$ 10.000.000,00",ACIMA DE 1 BILHAO DE REAIS,5 a 10
...,...,...,...,...,...,...,...,...,...,...
302888,COMERCIO VAREJISTA,COMERCIO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,OUTROS,EMPRESARIO INDIVIDUAL,True,"ATE R$ 81.000,00","DE R$ 81.000,01 A R$ 360.000,00",> 20
335955,COMERCIO VAREJISTA,COMERCIO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,OUTROS,EMPRESARIO INDIVIDUAL,True,"ATE R$ 81.000,00","DE R$ 81.000,01 A R$ 360.000,00",> 20
345211,COMERCIO VAREJISTA,COMERCIO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,OUTROS,EMPRESARIO INDIVIDUAL,True,"ATE R$ 81.000,00","DE R$ 81.000,01 A R$ 360.000,00",> 20
370607,COMERCIO VAREJISTA,COMERCIO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,OUTROS,EMPRESARIO INDIVIDUAL,True,"ATE R$ 81.000,00","DE R$ 81.000,01 A R$ 360.000,00",> 20


In [0]:
#Finally save it :)

'''
A == 1
0.8 <= B < 1
0.4 <= C < 0.8
0 <= D < 0.4
'''

# CLIENTE ONE
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_one_A.csv", index=False)
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_one_B.csv", index=False)
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_one_C.csv", index=False)
end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_one_D.csv", index=False)

# CLIENTE TWO
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_two_A.csv", index=False)
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_two_B.csv", index=False)
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_two_C.csv", index=False)
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_two_D.csv", index=False)

# CLIENTE THREE
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_three_A.csv", index=False)
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_three_B.csv", index=False)
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_three_C.csv", index=False)
# end.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/client_three_D.csv", index=False)


In [0]:
end_full = _market.iloc[end.index]

# CLIENTE ONE
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_one_A.csv", index=False)
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_one_B.csv", index=False)
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_one_C.csv", index=False)
end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_one_D.csv", index=False)

# CLIENTE TWO
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_two_A.csv", index=False)
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_two_B.csv", index=False)
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_two_C.csv", index=False)
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_two_D.csv", index=False)

# CLIENTE THREE
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_three_A.csv", index=False)
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_three_B.csv", index=False)
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_three_C.csv", index=False)
# end_full.to_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/3.Deployed/full_client_three_D.csv", index=False)